In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from time import sleep

%matplotlib inline

# N-step bandits
## $\epsilon$-greedy and UCB1

Here I adapt eps-greedy and ucb1 to a N-step bandits problem.

Each step $t$ has $n_t$ available actions. And there are $S_T = \prod_0^T n_t$ possible intermediate or final steps (or T-paths).

In [ ]:
def make_rngs(seed, n):
    """Makes n identical separate random generators"""
    return [np.random.default_rng(seed) for i in range(n)]

def simulate_rewards(rng, p_choice, p_opt_choice):
    """Simulates one trial for two bandits: chosen and optimal. Both are represented as probs to get reward +1."""
    x = rng.random()
    r = 1. if x <= p_choice else 0.
    r_opt = 1. if x <= p_opt_choice else 0.
    return r, r_opt

In [ ]:
# %%time
def test_eps_greedy(n_trials, bandits, eps=.01):
    rew_rng, eps_rng = make_rngs(seed, 2)
    
    # For each t, there're S_t = s_0 * s_1 *...*s_t paths. Shapes[t] - the shape of this t-dim array containing all possible t-paths.
    # Each path tau=(a_0, a_1,...a_t) has its own stats - Rs[t][tau] and Ns[t][tau]
    shapes = [bandits.shape[:i+1] for i in range(bandits.ndim)]
    Rs = [np.zeros(shape) for shape in shapes]
    Ns = [np.full(shape, 1e-5) for shape in shapes]
    optimal = np.max(bandits)
    
    def select_simulate_backprop(t, idx):
        # idx: t-1 path, so R and N are 1-dim vectors of size n_t (num of actions at t)
        R, N = Rs[t][idx], Ns[t][idx]
        n_actions = R.shape[0]
        Q = R/N
        action = eps_rng.choice(n_actions) if eps_rng.random() <= eps else np.argmax(Q)
        idx = idx + (action,)
        if t+1 < len(shapes):
            r, r_opt = select_simulate_backprop(t+1, idx)
        else:
            r, r_opt = simulate_rewards(rew_rng, bandits[idx], optimal)
        
        R[action] += r
        N[action] += 1
        return r, r_opt

    every_k = max(n_trials / 1000, 1)
    cum_regret, regret = 0, []
    
    for trial in range(1, n_trials+1):
        r, r_opt = select_simulate_backprop(0, ())
        
        cum_regret += r_opt - r
        if trial % every_k == 0:
            regret.append(cum_regret)        
        
    return regret, Rs[0], Ns[0]

seed = 1337
np.random.seed(seed)

# MDP states are declared from the end: 3 actions for t=0, 4 actions for t=1, 6 actions for t=2
bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = np.expand_dims(bandits, -1) * np.array([.35, .4, .44])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))

n_trials = 100000
for eps_perc in [5]:
    eps = eps_perc / 100.
    regret, R, N = test_eps_greedy(n_trials, bandits, eps)
    print(R/N, N.astype(np.int))
    _ = plt.plot(regret, label=f'$\epsilon$={eps_perc}%')
    
_ = plt.legend()

In [ ]:
# %%time
def test_ucb1(n_trials, bandits, eps=.01):
    rew_rng, exp_rng = make_rngs(seed, 2)
    
    # For each t, there're S_t = s_0 * s_1 *...*s_t paths. Shapes[t] - the shape of this t-dim array containing all possible t-paths.
    # Each path tau=(a_0, a_1,...a_t) has its own stats - Rs[t][tau] and Ns[t][tau]
    shapes = [bandits.shape[:i+1] for i in range(bandits.ndim)]
    Rs = [np.zeros(shape) for shape in shapes]
    Ns = [np.full(shape, 1e-5) for shape in shapes]
    optimal = np.max(bandits)
    
    def select_simulate_backprop(t, idx, trial):
        # idx: t-1 path, so R and N are 1-dim vectors of size n_t (num of actions at t)
        R, N = Rs[t][idx], Ns[t][idx]
        Q = R/N
        U = (2 * np.log(trial) / N)**.5
        action = np.argmax(Q + U)
        idx = idx + (action,)
        if t+1 < len(shapes):
            r, r_opt = select_simulate_backprop(t + 1, idx, trial)
        else:
            r, r_opt = simulate_rewards(rew_rng, bandits[idx], optimal)
        
        R[action] += r
        N[action] += 1
        return r, r_opt

    every_k = max(n_trials / 1000, 1)
    cum_regret, regret = 0, []
    
    for trial in range(1, n_trials):
        r, r_opt = select_simulate_backprop(0, (), trial)
        
        cum_regret += r_opt - r
        if trial % every_k == 0:
            regret.append(cum_regret)        
        
    return regret, Rs[0], Ns[0]

seed = 1337
np.random.seed(seed)

# MDP states are declared from the end: 3 actions for t=0, 4 actions for t=1, 6 actions for t=2
bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = np.expand_dims(bandits, -1) * np.array([.35, .4, .44])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))

n_trials = 100000
regret, R, N = test_ucb1(n_trials, bandits)
print(R/N, N.astype(np.int))
_ = plt.plot(regret, label='ucb1')


for eps_perc in [1, 5]:
    eps = eps_perc / 100.
    regret, R, N = test_eps_greedy(n_trials, bandits, eps)
    print(R/N, N.astype(np.int))
    _ = plt.plot(regret, label=f'$\epsilon$={eps_perc}%')

_ = plt.legend()

## UCB1 on SDR cells

Here I adapt MCTS with UCB1 policy to states represented as sets of cells. Each cell accumulate its own R and N statistics independently.

UCB1 for a state is calculated as an aggregate of UCB1 values of its cells, i.e. each action cell produces its own UCB1 value, and then they're accumulated (for now - as average).

In [ ]:
def init_cells(shape, n_cells, n_shared_cells):
    # shape == t-path
    rng = np.random.default_rng(seed)
    
    idx = int(np.prod(shape[:-1]))
    n_actions = shape[-1]
    
    # path is split into merged t-1 path and (n_t x n_cells). The last array of cells is split between actions at time t with n_shared_cells overlap
    shape = (idx, ) + (n_actions * n_cells, )
    R, N = np.zeros(shape), np.full(shape, 1e-5)

    # list of a_t arrays by (n_cells + n_shared_cells) indices. Each "row" - action cells indices
    action_cell_indices = []
    for j in range(idx):
        action_cell_indices.append([])
        for i in range(n_actions):
            base_indices = list(range(i*n_cells, (i+1)*n_cells))
            shared_indices = [
                x if x < i*n_cells else x + n_cells
                for x in rng.choice((n_actions - 1) * n_cells, n_shared_cells, replace=False)
            ]
            action_cell_indices[j].append(sorted(base_indices + shared_indices))
        action_cell_indices[j] = np.array(action_cell_indices[j])
    return R, N, np.array(action_cell_indices)

In [ ]:
def test_cells_ucb1(n_trials, bandits, n_cells, k_cells, n_shared_cells):
    rew_rng, exp_rng, upd_rng = make_rngs(seed, 3)
    
    shapes = [bandits.shape[:i+1] for i in range(bandits.ndim)]
    Rs, Ns, action_cell_indices = zip(*[init_cells(shape, n_cells, n_shared_cells) for shape in shapes])
    optimal, every_k, cum_regret, regret = np.max(bandits), max(n_trials / 1000, 1), 0, []
    
    def select_simulate_backprop(t, flat_idx, idx, trial):
        # idx: t-1 path; flat_idx: merged t-1 path
        R, N, cell_indices = Rs[t][flat_idx], Ns[t][flat_idx], action_cell_indices[t][flat_idx]
        Q = R/N
        U = ((k_cells / n_cells) * 2 * np.log(trial) / N)**.5
        
        n_actions = len(cell_indices)
        # V is a 2d arr, each row i has UCB1 values for a_i cells. Then they are averaged for each action.
        V = (Q + U)[cell_indices]        
        action = np.argmax(V.mean(axis=1))
        
        flat_idx, idx = flat_idx * n_actions + action, idx + (action, )
        if t+1 < len(shapes):
            r, r_opt = select_simulate_backprop(t + 1, flat_idx, idx, trial)
        else:
            r, r_opt = simulate_rewards(rew_rng, bandits[idx], optimal)
        
        # chooses k_cells from action cells to update
        upd_indices = upd_rng.choice(cell_indices[action], size=k_cells, replace=False)
        R[upd_indices] += r
        N[upd_indices] += 1
        return r, r_opt

    for trial in range(1, n_trials):
        r, r_opt = select_simulate_backprop(0, 0, (), trial)
        
        cum_regret += r_opt - r
        if trial % every_k == 0:
            regret.append(cum_regret) 
    
    # returns Q_0: values for the first step, i.e. Q(s_0, a_0) for each a_0^i
    Q_0 = Rs[0][0] / Ns[0][0]
    return regret, Q_0[action_cell_indices[0][0]].mean(axis=1), Ns[0][0][action_cell_indices[0][0]].mean(axis=1)

seed = 1337
np.random.seed(seed)

bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = np.expand_dims(bandits, -1) * np.array([.35, .4, .44])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))
print(bandits.shape)

n_trials = 100000
k = 4
for n_shared_cells in [0, 2, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')

n_shared_cells = 8
for k in [2, 4, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')


regret, R, N = test_ucb1(n_trials, bandits)
print(R/N, N.astype(np.int))
_ = plt.plot(regret, label='ucb1')


for eps_perc in [1, 5]:
    eps = eps_perc / 100.
    regret, R, N = test_eps_greedy(n_trials, bandits, eps)
    print(R/N, N.astype(np.int))
    _ = plt.plot(regret, label=f'$\epsilon$={eps_perc}%')

_ = plt.legend()

We see that UCB1 on cells performs quite similar to the simple UCB1 and better than $\epsilon$-greedy strategy even with overlapping action cells.

In [ ]:
seed = 1337
np.random.seed(seed)

bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = np.expand_dims(bandits, -1) * np.array([.35, .4, .44])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))
print(bandits.shape)

n_trials = 100000
k = 4
for n_shared_cells in [2, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')

n_shared_cells = 8
for k in [2, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')


regret, R, N = test_ucb1(n_trials, bandits)
print(R/N, N.astype(np.int))
_ = plt.plot(regret, label='ucb1')

_ = plt.legend()

Above is presented the similar experiment (without eps-greedy) to give a sence of how close to simple UCB1 is our approach.

And below is the experiment with overlapping. At each trial we update the whole action cells set (i.e. k = n_cells + n_shared_cells). The question is how good is our approach against overlapping? It resilient to a small overlapping, but as it grows over some threshold it start making much difference.

In [ ]:
seed = 1337
np.random.seed(seed)

bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = np.expand_dims(bandits, -1) * np.array([.35, .4, .44])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))
print(bandits.shape)

n_trials = 100000
k = 12
for n_shared_cells in [0, 6, 12, 18]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, k, k + n_shared_cells, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')

regret, R, N = test_ucb1(n_trials, bandits)
print(R/N, N.astype(np.int))
_ = plt.plot(regret, label='ucb1')


# for eps_perc in [1, 5]:
#     eps = eps_perc / 100.
#     regret, R, N = test_eps_greedy(n_trials, bandits, eps)
#     print(R/N, N.astype(np.int))
#     _ = plt.plot(regret, label=f'$\epsilon$={eps_perc}%')

_ = plt.legend()

And the last experiment - is for 2-step MDP instead of 3-step.

In [ ]:
seed = 1337
np.random.seed(seed)

bandits = np.array([.1, .15, .4, .5, .65, .7])
bandits = np.expand_dims(bandits, -1) * np.array([.3, .5, .8, .88])
bandits = bandits.T if bandits.ndim > 1 else bandits
print(bandits.reshape(bandits.shape[0], -1).max(axis=-1))
print(bandits.reshape(bandits.shape[0], -1).mean(axis=-1))
print(bandits.shape)

n_trials = 100000
k = 4
for n_shared_cells in [2, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')

n_shared_cells = 8
for k in [2, 8, 16]:
    regret, probs, cnts = test_cells_ucb1(n_trials, bandits, 12, k, n_shared_cells)
    print(probs, cnts.astype(np.int))
    _ = plt.plot(regret, label=f'{k}-{n_shared_cells}')


regret, R, N = test_ucb1(n_trials, bandits)
print(R/N, N.astype(np.int))
_ = plt.plot(regret, label='ucb1')

_ = plt.legend()